# 爬蟲-網頁資料擷取

## 概念

### 網頁的組成



- HTML + CSS + JavaScritp
  - HTML: 定義網頁的內容、結構
  - CSS: 顯示的風格Style
  - JS: 行為...

- HTML是階層式文件結構，由許多元素(Elements)組成
- 一個元素包含開始標籤、結束標籤、屬性及內容

`<Tag 屬性>內容</Tag>`

#### 常用標籤

|標籤名稱|用途|
-|-
`<h1> ~<h6> `| 標題
`<p>`|段落
`<a href="https://www.123.com">`|超連結
\<table\>|表格
\<tr\>|表格內的row
\<td\>|表格內的cell
\<br/\>|換行(無結束標籤)

#### 常用屬性(Attributes)

|屬性名稱|用途|
-|-
class|標籤的類別(可重複)
id|標籤的id(不可重複)
title|標籤的顯示資訊
style|標籤的樣式
data-*|自行定義的屬性

### 擷取網頁必要知識

- 在HTTP協定中，定義了多種不同的method做為服務的請求方法，近年來由於行動裝置的普及化，越來越多的產品及網站都提供了WebAPI服務，既然我們要擷取網頁內容，就必須知道對HTTP請求方式。
- 在HTTP 1.1的版本中定義了八種 Method (方法)，如下所示：
    - OPTIONS
    - **GET**
    - HEAD
    - **POST**
    - PUT
    - DELETE
    - TRACE
    - CONNECT

- 最常見的method為以下5種:

|method|意義|
|-|- |
|GET|取得(想要的服務)的資料或是狀態。|
|POST|如同填表般的行為，以新增一項資料。
|PUT|利用更新的方式於"指定位置"新增一項資料。
|PATCH|在現有的資料欄位中，增加或部分更新一筆新的資料。
|DELETE|刪除指定資料。

- 更進一步了解請參閱W3C制定規範[RFC 5789](https://www.w3.org/Protocols/rfc2616/rfc2616-sec9.html#sec9.5)
- [淺談 HTTP Method：表單中的 GET 與 POST 有什麼差別？](https://blog.toright.com/posts/1203/%E6%B7%BA%E8%AB%87-http-method%EF%BC%9A%E8%A1%A8%E5%96%AE%E4%B8%AD%E7%9A%84-get-%E8%88%87-post-%E6%9C%89%E4%BB%80%E9%BA%BC%E5%B7%AE%E5%88%A5%EF%BC%9F.html)

- 另外在網頁與資料庫的操作過程中，也會經常聽到CRUD這個詞，CRUD是指 新增(Create)、讀取(Read)、更新(Update)、刪除(Delete)的主要4個操作資料庫(如MySQL等)常用的功能

- 參閱[Day 15 - 實作第一個 CRUD 之 Create、Update、Delete](https://ithelp.ithome.com.tw/articles/10206716)

### 淺談Restful API


- REST全名 Resource Representational State Transfer，可譯為具象狀態傳輸，其核心精神在於借用 HTTP 協定做為基礎，讓API規格簡單一致:
    - Resource：資源。
    - Representational：表現形式，如JSON，XML．．．
    - State Transfer：狀態變化。即上述講到的可利用HTTP動詞們來做呼叫。

- REST指的是網路中Client端和Server端的一種呼叫服務形式，透過既定的規則，滿足約束條件和原則的應用程式設計，對資源的操作包括獲取、創建、修改和刪除資源，可對應資料庫基本操作：新增、讀取、更新、刪除(Create、Read、Update、Delete, **CRUD**)。

- 舉例商品WebAPI的interface：
    - 獲得商品資料 GET   /getItem/9527
    - 新增商品資料 POST  /createItem
    - 更新商品資料 POST  /updateItem/
    - 刪除商品資料 POST  /deleteItem/

- 運用RESTful API 開發的WebAPI的interface:
    - 獲取商品資料 /GET/items/9527
    - 新增商品資料 /POST/items
    - 更新商品資料 /PATCH/items/9527
    - 刪除商品資料 /DELETE/items/9527

- 即便有些離題，但增加網頁常識對蒐集真實世界資料總有助益。

- 延伸閱讀
    - [[不是工程師] 休息(REST)式架構- 寧靜式(RESTful)的Web API是現在的潮流？](https://progressbar.tw/posts/53)
    >當Web service使用Web API進行介面介接時，每一串我們設計的URL，就會是一個專屬的服務『窗口』。
    - [RESTful API 設計準則與實務經驗](https://blog.toright.com/posts/5523/restful-api-%E8%A8%AD%E8%A8%88%E6%BA%96%E5%89%87%E8%88%87%E5%AF%A6%E5%8B%99%E7%B6%93%E9%A9%97.html)

### 什麼是網路爬蟲(Web Crawler)

![](https://miro.medium.com/max/1132/1*YfeP5WFbn0MwI76kuTM38w.png)
https://blog.apify.com/what-is-web-scraping-1b548f8d6ac1


- 網路爬蟲像是機器人，自動化的幫你擷取目標資訊
- 爬蟲無所不在，谷哥(度娘?)都是

- 爬蟲應用?
  - 熱門遊戲評論、輿論分析系統、銷售分析、旅遊訂票...


- 再看一次網頁元素結構
  - `<p class="value"> target </p>`
  - `<目標標籤+輔助資訊>目標資訊</目標標籤>`

- 寫爬蟲之前要注意的
  - 有沒有人寫過?
  - 該網站是否已經有API供人取用?
  - 要有禮貌(大量、頻繁的請求會造成伺服器負荷)

### 爬蟲的主要步驟

- 取得指定的HTML資料
  - 你有Python的requests模組可以取得HTML
- 解析資料取得目標資訊
  - 你有Python的 BeautifulSoup模組可以解析HTML
- 自動化(Robotic Process Automation, RPA)串起你的服務

### 有禮貌的爬蟲

- 爬取網站資料時，請勿過於頻繁的索取資料，善用time.sleep()


In [1]:
import time

print('----start----')
time.sleep(3)
print('----done----')


----start----
----done----


In [3]:
import time
import random

random_s = 1 + random.randint(0,2) #加入隨機秒數
print('----start----')
time.sleep( random_s)
print('----done----')


----start----
----done----


- 經過SEO的網站可能有允許/禁止爬取的頁面規範，可至該網站網域`https://*.*.*/robots.txt`查看，如`https://www.facebook.com/robots.txt`及`https://twitter.com/robots.txt`
- `robots.txt`只是表明不要到網站這些地方，許多web scraping工具會遵循（但也可關掉預設值）

- 另外也請注意智慧財產權(Intellectual Property, IP)相關的類型，如商標、著作權、專利，如果有未獲同意、實際傷害及故意，則有觸法之虞。


- 為了避免頻繁請求被目標伺服器阻擋，測試爬蟲時可採用你的手機(4g)網路，如果被ben，手機改飛航模式一陣子再開4g網路，即會在自動分配(取得)新的IP Address

## 開始動手做GET網頁


### 以example網頁為例


- 先觀察目標網頁: http://www.example.com/
- 以`requests.get`抓取網頁原始碼，並輸出結果

In [8]:
import requests


res = requests.get('http://www.example.com/')
print(res.text[:500])

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
    


### Requests常用函數


- `response.status_code`
  - 200 OK
  - 403 Forbidden （禁止）
  - 404 Not Found
- `response.encoding`
  - 如果是中文網站要特別注意編碼的問題
  - 常用編碼UTF-8，windows可能會遇到CP950、Big5等編碼問題
- response.text
  - 目標網頁的HTML文字，即被Tag包起來的內文(目標資訊）


In [9]:
res.encoding

'UTF-8'

### 以Beautiful Soup讀取並解析HTML


- [Beautiful Soup 4.2.0 文档](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)
- Beautiful Soup是強大的HTML解析器
- 創建一個 BeautifulSoup 物件，將網頁讀入。
- 此時soup的datatype為bs4.BeautifulSoup物件，此物件中包含了整個 HTML 文件的結構樹，有了這個結構樹之後，就可以輕鬆找出任何有興趣的資料了。
- 如尚未安BeautifulSoup4模組，在筆記本環境可使用`
!pip3 install beautifulsoup4`指令安裝，非筆記本的CMD環境去掉驚嘆號。

下表列出了主要的解析器，以及它們的優缺點：

解析器|使用方法|優勢|	劣勢|
-|-|-|-
Python標準庫|	BeautifulSoup(markup,"html.parser")	|Python的內建標準庫、執行速度適中、文檔容錯能力強|Python 2.7.3及3.2.2之前的版本中文檔容錯能力差
lxml HTML 解析器|	BeautifulSoup(markup, "lxml")	|速度快、文檔容錯能力強(通常用這個)|需要安装C语言库
xml XML 解析器|BeautifulSoup(markup, "xml")|速度快、唯一支持XML的解析器|需要安装C语言库
html5lib	|BeautifulSoup(markup, "html5lib")	|最好的容錯性、以瀏覽器的方式解析文檔、生成HTML5格式的文檔|速度慢、不依賴外部擴展


In [10]:
from bs4 import BeautifulSoup 

soup = BeautifulSoup(res.text, "lxml")

type(soup)

bs4.BeautifulSoup

- 輸出排版後的 HTML 程式碼

In [11]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Example Domain
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <style type="text/css">
   body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
  </style>
 </head>
 <body>
  <div>
   <h1>
    Example Domain
   </h1>
   <p>
    This dom

#### BeautifulSoup的常用函數
- `soup.find()` 找一個標籤 tag
-  回傳第一個被tag包圍的區塊
- 傳入的引數第一個通常是 tag 名稱，第二個引數若未指明屬性就代表 class 名稱，也可以直接使用 id 等屬性去定位區塊。定位到區塊後，可以取出其屬性與包含的字串值

In [0]:
?soup.find()
#soup.find(name=None, attrs={}, recursive=True, text=None, **kwargs)

In [13]:
soup.find('p')

<p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>

- soup.find_all() 回傳全被tag包圍的區塊，回傳為list

In [14]:
soup.find_all('p')

[<p>This domain is for use in illustrative examples in documents. You may use this
     domain in literature without prior coordination or asking for permission.</p>,
 <p><a href="https://www.iana.org/domains/example">More information...</a></p>]

In [15]:
a = soup.find("a")
a

<a href="https://www.iana.org/domains/example">More information...</a>

In [16]:
a["href"]

'https://www.iana.org/domains/example'

In [17]:
a.text

'More information...'

- 取得節點文字內容

In [19]:
title_tag = soup.title
title_tag

<title>Example Domain</title>

In [21]:
title_tag.string

'Example Domain'

- 取出節點屬性
  - 若要取出 HTML 節點的各種屬性，可以使用 `get`。
  - 如果不用`get`也可以擷取屬性，但不存在時會出現錯誤，有礙後續爬蟲執行。
  - 使用`get`如無此屬性，回傳結果為none。
  - 其他詳細用法可參考 [BeautifulSoup的官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [0]:
# 會顯示錯誤的例子
soup.find("p")['style']      

In [25]:
# 使用get()，未搜尋到的結果回傳為None
s = soup.find('p').get('style')
print(s)

None


In [27]:
s2 = soup.find('a').get('href')
print(s2)

https://www.iana.org/domains/example


- 延伸閱讀:
    - [給初學者的 Python 網頁爬蟲與資料分析 (3) 解構並擷取網頁資料](http://blog.castman.net/%E6%95%99%E5%AD%B8/2016/12/22/python-data-science-tutorial-3.html)
    - [Python 使用 Beautiful Soup 抓取與解析網頁資料，開發網路爬蟲教學](https://blog.gtwang.org/programming/python-beautiful-soup-module-scrape-web-pages-tutorial/)

In [30]:
# 搜尋節點
p_tags = soup.find_all("p")
p_tags

[<p>This domain is for use in illustrative examples in documents. You may use this
     domain in literature without prior coordination or asking for permission.</p>,
 <p><a href="https://www.iana.org/domains/example">More information...</a></p>]

In [31]:
# 搜尋節點並從list取出內容
p_tags = soup.find_all("p")

for tag in p_tags:
  print(tag.string)

This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.
More information...


In [34]:
# 取出節點屬性
a_tags = soup.find_all("a")

for tag in a_tags:
  print(tag['href'] )

https://www.iana.org/domains/example


#### 以list同時搜尋多種標籤

In [36]:
# 搜尋所有超連結與粗體字
tags = soup.find_all(["a", "b", "p" ,"div"])
print(tags)

[<div>
<h1>Example Domain</h1>
<p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>
<p><a href="https://www.iana.org/domains/example">More information...</a></p>
</div>, <p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>, <p><a href="https://www.iana.org/domains/example">More information...</a></p>, <a href="https://www.iana.org/domains/example">More information...</a>]


#### `find_all()`以`limit`參數限制搜尋數量
- 只有1個就可以改為`find()`

In [37]:
# 限制搜尋結果數量
tags = soup.find_all(["a", "b"], limit=2)
print(tags)

[<a href="https://www.iana.org/domains/example">More information...</a>]


#### 限制`find_all()`關閉遞迴搜尋
- 預設find_all()會遞迴搜尋所有子節點
- 以`recursive=False` 關閉遞迴搜尋功能

In [38]:
# 不使用遞迴搜尋，僅尋找次一層的子節點
soup.html.find_all("title", recursive=False)

[]

In [0]:
# 不指定標籤，但找出所有屬性 class = "zzz" 的標籤 
print(soup.find_all("", {"class":"zzz"}))

In [0]:
# 找出所有 td 標籤的第三個並找出其中的 a 標籤 
print(soup.find_all("td")[2].find("a")) 

In [0]:
# 找出所有內容等於 Example Domain 的文字 
print(soup.find_all(text="Example Domain"))

In [0]:
# 找出第一個 a 標籤並印出屬性 
print(soup.find("a").attrs) 
print(soup.find("a")["href"]) 

In [0]:
#找出所有 td 標籤，並用 len 計算長度 
print(len(soup.find_all("a")))

# 找到 div 標籤，屬性 id = "id1"，再印出其內容 
print(soup.find("div", id="id1").text)
# 透過觀察網頁可以發現 列3欄3 有個 id = hyperlink 可 以幫助我們定位這個 tag，再把 tag 的 href 找出來 print(soup.find("a", {"id":"hyperlink"})["href"])

### 結合正規表達式regular expression進行搜尋


- 正規表達式對於精準抓取網頁的各種標籤及內文非常有幫助，解決了許多Xpath與CSS selector無法精確擷取的問題，有必要好好理解。
- 擷取的文句段落可以使用[regex101.com](https://regex101.com/)測試。

|意義|表示|範例|
|-|-|-|
|Start|`^`|123ABC `/^1/`
|End|`$`|123ABC `/5$/`
|Range|`[<Start>-End>]`|123ABC `/^[0-2]/`
|Number|`\d`|123ABC `/^\d/`
|Character|`\w`|123ABC `/\w$/`
|Invisible Character|`\s`|`Tab, Space, Escape, …`
|Zero or One|`+`|
|Zero or More|`*`|123ABC `/\w+$/`
|One or More|`?`|123ABC `/[0-2]/`
|Named Group|`(?P<name>expression)`|
|Named Group|`(?<name>expression)`|


#### Python的re模組
- 推薦使用re.findall()
- 可至[regex101](https://regex101.com/)嘗試

##### 參考寫法
```python
import re

# 找出所有內容等於 python_crawler 的文字 
pattern = "我寫好的 regular expression" 
string = "我想要找的字串" 
re.findall(pattern, string)
```

In [40]:
import re

# 找出所有內容等於 python_crawler 的文字 
pattern = "的" 
string = "我想要找的字串"  #resquests.text也是字串
re.findall(pattern, string)

['的']

In [45]:
import re

# 找出html裡的超連結 
pattern = r'href=\"(.*)\"|href=\'(.*)\'' #參閱https://regex101.com/r/uw6MLH/1
string = res.text
re.findall(pattern, string)[0][0]

'https://www.iana.org/domains/example'

##### 更多re

In [0]:
import re

let re = /<Pattern>/;

# Find First Match
match = re.search(<Pattern>, <String>)

let match = re.exec(<String>);

# Find All Matches
match = re.findall(<Pattern>, <String>)

# Get Matched Groups
match.group(<Index>)
match.group(<Name>)

# Get Matched Groups
match[<Index>]
match.groups[<Name>]

#Split
re.split(<Pattern>, <String>)
# Replace
re.sub(<Pattern>, <Replace>, <String>)

## 網頁擷取實例


### 以PPT 為例


- 這邊開始要示範使用Chrome開發者工具進行搜尋
- 先觀察目標網頁: https://www.ptt.cc/bbs/StupidClown/index.html
- 使用Chrome瀏覽器，以滑鼠右鍵選擇「檢查」，快捷鍵在windows環境為ctrl+Shift+I或F12

- 另外如果要用別人寫好的，參閱https://dotblogs.com.tw/codinghouse/2018/10/22/pttcrawler

![](https://i.imgur.com/K55v4SH.png)


- 文章列表可以觀察到推文數、文章標題、作者、日期及文章連結
- 我們先觀察他的樹狀結構，對應的標籤與屬性
- 以COPY XPath紀錄

|名稱|selector|
-|-
標題|`//*[@id="main-container"]/div[2]/div[4]/div[2]/a`
連結|`//*[@id="main-container"]/div[2]/div[4]/div[2]/a`

In [0]:
#目標網址https://www.ptt.cc/bbs/StupidClown/index.html
import requests
from bs4 import BeautifulSoup

res = requests.get('https://www.ptt.cc/bbs/StupidClown/index.html')
soup = BeautifulSoup(res.text ,"lxml")
print(res.text[:500])

In [0]:
print(soup.prettify())

- 有抓到網頁，接下來如果簡單針對連結、標題的話，觀察都在div標籤的class='title'裡

In [50]:
results = soup.select("div.title")
print(results)

[<div class="title">
<a href="/bbs/StupidClown/M.1572014689.A.F9B.html">[無言] 誰會委託同事買餐巾紙?</a>
</div>, <div class="title">
<a href="/bbs/StupidClown/M.1572064384.A.96C.html">Fw: [問卦] 鮭魚握壽司被蒸熟了該怎麼辦？</a>
</div>, <div class="title">
<a href="/bbs/StupidClown/M.1572068103.A.43A.html">[無言] 牙線棒卡在牙套上</a>
</div>, <div class="title">
<a href="/bbs/StupidClown/M.1158735717.A.828.html">[公告] 笨板板規</a>
</div>, <div class="title">
<a href="/bbs/StupidClown/M.1435710970.A.31E.html">[公告]本板即日起不可PO問卷文</a>
</div>, <div class="title">
<a href="/bbs/StupidClown/M.1569938128.A.51D.html">[公告] 10月份置底閒聊文</a>
</div>]


In [51]:
article_href = soup.select("div.title a")
article_href

[<a href="/bbs/StupidClown/M.1572014689.A.F9B.html">[無言] 誰會委託同事買餐巾紙?</a>,
 <a href="/bbs/StupidClown/M.1572064384.A.96C.html">Fw: [問卦] 鮭魚握壽司被蒸熟了該怎麼辦？</a>,
 <a href="/bbs/StupidClown/M.1572068103.A.43A.html">[無言] 牙線棒卡在牙套上</a>,
 <a href="/bbs/StupidClown/M.1158735717.A.828.html">[公告] 笨板板規</a>,
 <a href="/bbs/StupidClown/M.1435710970.A.31E.html">[公告]本板即日起不可PO問卷文</a>,
 <a href="/bbs/StupidClown/M.1569938128.A.51D.html">[公告] 10月份置底閒聊文</a>]

In [53]:
# 逐一取出標題、合併超連結
for a in article_href:
  print('title:', a.text)
  print('href:','https://www.ptt.cc'+a['href'])

  #打開連結內的網頁並另存
  content_url = 'https://www.ptt.cc'+ a['href']
  r = requests.get(content_url)
  with open ( a.text + '.html', 'w+') as f:
    f.write(r.text)
    print('saved')

title: [無言] 誰會委託同事買餐巾紙?
href: https://www.ptt.cc/bbs/StupidClown/M.1572014689.A.F9B.html
saved
title: Fw: [問卦] 鮭魚握壽司被蒸熟了該怎麼辦？
href: https://www.ptt.cc/bbs/StupidClown/M.1572064384.A.96C.html
saved
title: [無言] 牙線棒卡在牙套上
href: https://www.ptt.cc/bbs/StupidClown/M.1572068103.A.43A.html
saved
title: [公告] 笨板板規
href: https://www.ptt.cc/bbs/StupidClown/M.1158735717.A.828.html
saved
title: [公告]本板即日起不可PO問卷文
href: https://www.ptt.cc/bbs/StupidClown/M.1435710970.A.31E.html
saved
title: [公告] 10月份置底閒聊文
href: https://www.ptt.cc/bbs/StupidClown/M.1569938128.A.51D.html
saved


In [54]:
%ls

'Fw: [問卦] 鮭魚握壽司被蒸熟了該怎麼辦？.html'
 sample_data/
'[公告] 10月份置底閒聊文.html'
'[公告]本板即日起不可PO問卷文.html'
'[公告] 笨板板規.html'
'[無言] 牙線棒卡在牙套上.html'
'[無言] 誰會委託同事買餐巾紙?.html'


In [0]:
#需滿18歲要加cookies
import requests

def fetch(url):
    response = requests.get(url)
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
resp = fetch(url)  # step-1

print(resp.text) # result of setp-1

- 更多可參考[爬蟲教學 CrawlerTutorial](https://github.com/leVirve/CrawlerTutorial)

### 以wiki亞洲國家資訊為例

- 參考來源[Web Scraping Wikipedia Tables using BeautifulSoup and Python](https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722)

In [0]:
import requests

website_url = requests.get('https://en.wikipedia.org/wiki/\
                           List_of_Asian_countries_by_area').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

![](https://miro.medium.com/max/740/1*NyaaGqqHnemKSWu8DQqUHQ.png)

In [57]:
My_table = soup.find("table",{"class":"wikitable sortable"})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Rank
</th>
<th>Country
</th>
<th>Area (km²)
</th>
<th class="unsortable">Notes
</th></tr>
<tr>
<td>1
</td>
<td><span class="flagicon" style="display:inline-block;width:25px;"><img alt="" class="thumbborder" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/en/thumb/f/f3/Flag_of_Russia.svg/23px-Flag_of_Russia.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/f3/Flag_of_Russia.svg/35px-Flag_of_Russia.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/f3/Flag_of_Russia.svg/45px-Flag_of_Russia.svg.png 2x" width="23"/></span> <a href="/wiki/Russia" title="Russia">Russia</a>*
</td>
<td>13,100,000
</td>
<td>17,098,242 including European part<sup class="reference" id="cite_ref-russiaTotalAreaByCIA_1-0"><a href="#cite_note-russiaTotalAreaByCIA-1">[1]</a></sup>
</td></tr>
<tr>
<td>2
</td>
<td><span class="flagicon" style="display:inline-block;width:25px;"><img alt=""

In [58]:
links = My_table.findAll('a')
links

[<a href="/wiki/Russia" title="Russia">Russia</a>,
 <a href="#cite_note-russiaTotalAreaByCIA-1">[1]</a>,
 <a href="/wiki/China" title="China">China</a>,
 <a href="/wiki/Hong_Kong" title="Hong Kong">Hong Kong</a>,
 <a href="/wiki/Macau" title="Macau">Macau</a>,
 <a href="/wiki/India" title="India">India</a>,
 <a href="#cite_note-2">[2]</a>,
 <a href="/wiki/Kazakhstan" title="Kazakhstan">Kazakhstan</a>,
 <a href="/wiki/Saudi_Arabia" title="Saudi Arabia">Saudi Arabia</a>,
 <a href="/wiki/Iran" title="Iran">Iran</a>,
 <a href="/wiki/Mongolia" title="Mongolia">Mongolia</a>,
 <a href="/wiki/Indonesia" title="Indonesia">Indonesia</a>,
 <a href="/wiki/Pakistan" title="Pakistan">Pakistan</a>,
 <a href="/wiki/Turkey" title="Turkey">Turkey</a>,
 <a href="/wiki/Myanmar" title="Myanmar">Myanmar</a>,
 <a href="/wiki/Afghanistan" title="Afghanistan">Afghanistan</a>,
 <a href="/wiki/Yemen" title="Yemen">Yemen</a>,
 <a href="/wiki/Thailand" title="Thailand">Thailand</a>,
 <a href="/wiki/Turkmenistan" t

In [0]:
Country = [ link.get('title') for link in links  if link.get('title') != None]

In [66]:
Country

['Russia',
 'China',
 'Hong Kong',
 'Macau',
 'India',
 'Kazakhstan',
 'Saudi Arabia',
 'Iran',
 'Mongolia',
 'Indonesia',
 'Pakistan',
 'Turkey',
 'Myanmar',
 'Afghanistan',
 'Yemen',
 'Thailand',
 'Turkmenistan',
 'Uzbekistan',
 'Iraq',
 'Japan',
 'Vietnam',
 'Malaysia',
 'Oman',
 'Philippines',
 'Laos',
 'Kyrgyzstan',
 'Syria',
 'Golan Heights',
 'Cambodia',
 'Bangladesh',
 'Nepal',
 'Tajikistan',
 'North Korea',
 'South Korea',
 'Jordan',
 'Azerbaijan',
 'United Arab Emirates',
 'Georgia (country)',
 'Sri Lanka',
 'Egypt',
 'Bhutan',
 'Taiwan',
 'Armenia',
 'Kuwait',
 'East Timor',
 'Qatar',
 'Lebanon',
 'Israel',
 'State of Palestine',
 'Brunei',
 'Singapore',
 'Bahrain',
 'Maldives']

In [67]:
import pandas as pd

df = pd.DataFrame()
df['Country'] = Country
df

,Country
0,Russia
1,China
2,Hong Kong
3,Macau
4,India
5,Kazakhstan
6,Saudi Arabia
7,Iran
8,Mongolia
9,Indonesia


In [0]:
df = df.sort_values(by="Country")
df.reset_index(drop = True)
df

## 練習

###  練習1

- 試著看懂並執行、拆解以下程式
- 程式來源https://github.com/jwlin/web-crawler-tutorial/blob/master/ch3/ptt_gossiping.py

In [78]:
import requests
import time
import json
from bs4 import BeautifulSoup


PTT_URL = 'https://www.ptt.cc'


def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html5lib')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').text.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)  # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2', ...
                    # 若不是, 不做任何事，push_count 保持為 0
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').text
                author = ''  # author = d.find('div', 'author').text if d.find('div', 'author') else ''
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count,
                    'author': author
                })
    return articles, prev_url


def get_author_ids(posts, pattern):
    ids = set()
    for post in posts:
        if pattern in post['author']:
            ids.add(post['author'])
    return ids

if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Gossiping/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        today = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, today)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, today)

        # 印出所有不同的 5566 id
        # print(get_author_ids(articles, '5566'))

        # 儲存或處理文章資訊
        print('今天有', len(articles), '篇文章')
        threshold = 50
        print('熱門文章(> %d 推):' % (threshold))
        for a in articles:
            if int(a['push_count']) > threshold:
                print(a)
        with open('gossiping.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

今天有 776 篇文章
熱門文章(> 50 推):
{'title': '[新聞] 星宇航空首架機交付 張國煒親自駕回台', 'href': '/bbs/Gossiping/M.1572072025.A.F0A.html', 'push_count': 57, 'author': ''}
{'title': '[新聞] 同志遊行前夕 蘇貞昌：相互尊重讓台灣更', 'href': '/bbs/Gossiping/M.1572070242.A.738.html', 'push_count': 58, 'author': ''}
{'title': '[新聞] 違法三缺一？中國觸怒民怨的「麻將館禁令」', 'href': '/bbs/Gossiping/M.1572068601.A.93D.html', 'push_count': 60, 'author': ''}
{'title': '[新聞] 進台北市區注意！明天同志遊行中午起交通', 'href': '/bbs/Gossiping/M.1572066682.A.275.html', 'push_count': 54, 'author': ''}
{'title': '[新聞] 非洲豬瘟肆虐 菲律賓養豬業每月損失約6億', 'href': '/bbs/Gossiping/M.1572066826.A.A34.html', 'push_count': 80, 'author': ''}
{'title': '[問卦] 有沒有做出PttChrome計算推樓插件的八卦', 'href': '/bbs/Gossiping/M.1572066384.A.30B.html', 'push_count': 99, 'author': ''}
{'title': '[爆卦] 中國食品凍漲擋不住了', 'href': '/bbs/Gossiping/M.1572065077.A.9C0.html', 'push_count': 99, 'author': ''}
{'title': '[新聞] 貧窮線調高！ 北市月收不到2萬4293元就', 'href': '/bbs/Gossiping/M.1572064119.A.A22.html', 'push_count': 55, 'author': ''}
{'title': '[爆卦] 

### 練習2

- 試著看懂並執行、拆解以下程式
- 程式來源https://github.com/jwlin/web-crawler-tutorial/blob/master/ch3/yahoo_movie.py

In [77]:
import requests
import re
import json
from bs4 import BeautifulSoup


Y_MOVIE_URL = 'https://tw.movies.yahoo.com/movie_thisweek.html'

# 以下網址後面加上 "/id=MOVIE_ID" 即為該影片各項資訊
Y_INTRO_URL = 'https://tw.movies.yahoo.com/movieinfo_main.html'  # 詳細資訊
Y_PHOTO_URL = 'https://tw.movies.yahoo.com/movieinfo_photos.html'  # 劇照
Y_TIME_URL = 'https://tw.movies.yahoo.com/movietime_result.html'  # 時刻表


def get_web_page(url):
    resp = requests.get(url)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_movies(dom):
    soup = BeautifulSoup(dom, 'html5lib')
    movies = []
    rows = soup.find_all('div', 'release_info_text')
    for row in rows:
        movie = dict()
        movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        movie['movie_id'] = get_movie_id(row.find('div', 'release_movie_name').a['href'])
        movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        movie['release_date'] = get_date(row.find('div', 'release_movie_time').text)
        movie['intro'] = row.find('div', 'release_text').text.replace(u'詳全文', '').strip()
        trailer_a = row.find_next_sibling('div', 'release_btn color_btnbox').find_all('a')[1]
        movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs.keys() else ''
        movies.append(movie)
    return movies


def get_date(date_str):
    # e.g. "上映日期：2017-03-23" -> match.group(0): "2017-03-23"
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    if match is None:
        return date_str
    else:
        return match.group(0)


def get_movie_id(url):
    # 20180515: URL 格式有變, e.g., 'https://movies.yahoo.com.tw/movieinfo_main/%E6%AD%BB%E4%BE%8D2-deadpool-2-7820.html
    # e.g., "https://tw.rd.yahoo.com/referurl/movie/thisweek/info/*https://tw.movies.yahoo.com/movieinfo_main.html/id=6707"
    #       -> match.group(0): "/id=6707"
    try:
        movie_id = url.split('.html')[0].split('-')[-1]
    except:
        movie_id = url
    return movie_id


def get_trailer_url(url):
    # e.g., 'https://tw.rd.yahoo.com/referurl/movie/thisweek/trailer/*https://tw.movies.yahoo.com/video/美女與野獸-最終版預告-024340912.html'
    return url.split('*')[1]


def get_complete_intro(movie_id):
    page = get_web_page(Y_INTRO_URL + '/id=' + movie_id)
    if page:
        soup = BeautifulSoup(page, 'html5lib')
        infobox = soup.find('div', 'gray_infobox_inner')
        print(infobox.text.strip())


def main():
    page = get_web_page(Y_MOVIE_URL)
    if page:
        movies = get_movies(page)
        for movie in movies:
            print(movie)
            # get_complete_intro(movie["movie_id"])
        with open('movie.json', 'w', encoding='utf-8') as f:
            json.dump(movies, f, indent=2, sort_keys=True, ensure_ascii=False)


if __name__ == '__main__':
    main()

{'expectation': '90%', 'ch_name': '雙子殺手', 'eng_name': 'Gemini Man', 'movie_id': '10017', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/October2019/YSeFXqDiSK0XrIcSdHi1-486x720.JPG', 'release_date': '2019-10-23', 'intro': '威爾史密斯飾演的一名頂尖殺手亨利布羅根，卻被一位神秘的年輕殺手追殺，而且這位年輕的殺手竟然能事先預測亨利的一舉一動。《雙子殺手》由金像獎得主李安執導，知名製作人傑瑞布洛克海默、大衛艾利森、戴娜戈柏和唐葛蘭傑製作。其他演員還有瑪麗伊莉莎白文斯蒂德、克里夫歐文和班奈狄克王。', 'trailer_url': 'https://movies.yahoo.com.tw/video/%E9%9B%99%E5%AD%90%E6%AE%BA%E6%89%8B-%E6%9C%80%E6%96%B0%E9%A0%90%E5%91%8A-035402781.html?movie_id=10017'}
{'expectation': '88%', 'ch_name': '電影版 吹響吧！上低音號～誓言的終章～', 'eng_name': 'Sound! Euphonium, the Movie -Our Promise: A Brand New Day-', 'movie_id': '10306', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/October2019/ZXplEx71mOQQlcTpxiYd-992x1418.JPG', 'release_date': '2019-10-23', 'intro': '順利在去年的全日本管樂競演會中出場的北宇治高中管樂社，升上二年級的黃前久美子和三年級的加部友惠，兩人開始一起負責指導從四月開始新加入的一年級新生。\n\xa0\n由於身為全國大賽的出場學校，因此吸引了大量的一年級新生入部，其中，有四名新生來到了低音部，包括乍看之下似乎毫無問題的久石奏

### 練習3


- 擷取並parse「批批踢JOKE版的一篇文章」
- 請依下列步驟練習：
    - 以GET方法將網頁https://www.ptt.cc/bbs/joke/M.1571755669.A.663.html 原始碼讀入
    - 依照上述步驟parse出推文內容及推文者
    - 透過for迴圈，整齊印出